# Project Simulation Method: Quadratic Generator

This project consists on using the simulation method to randomly generate the coefficient of the quadratic equation to match each case. The random coefficient is generated by using the **Inversion Method** by taking the set and each assigned value of probability correspond to each number.

This project has been built inside the python, and the javascript app.

In python, we need to run **pip install -r requirements.txt** to setup the package dependencies and then we can just run the **python main.py** to startup the application. On the other hand, we have the website deployed [Project MESIM](https://mesim.sokhengdin.dev)

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [3]:
# HELPER AND UTILITY FUNCTION
# Calculate reduce sum over finite
def calculate_reduce_sum(values):
    sum = 0.0
    
    for i in range(len(values)):
        sum += values[i]

    return float(sum)

# Manaully Calculate Mean E[X] = sum(xpx)
def calculate_expectation(weights, values):
    average = 0.0
    for i in range(len(values)):
        average += weights[i]*values[i]

    return average

# Manually calculate variance Var(X) = E[X**2] - E[X]**2
def calculate_variance(weights, values):
    E_X  = calculate_expectation(weights, values)
    E_X2 = calculate_expectation(weights, [v**2 for v in values])
    return E_X2 - E_X**2

# Generally generate discrete samples, with inversion method
def generate_discrete_sample(values, probs, N=1):
    cdf = []
    cumul = 0.0
    for p in probs:
        cumul += p
        cdf.append(cumul)
    samples = []
    for _ in range(N):
        U = np.random.rand()
        k = 0
        while k < len(values) - 1 and U > cdf[k]:
            k += 1
        samples.append(values[k])
    if N == 1:
        return samples[0]
    return np.array(samples)

# TO display equation
def display_equation(a, b, c, delta):
    a = round(a, 4)
    b = round(b, 4)
    c = round(c, 4)
    delta = round(delta, 4)

    def fmt(coef, var, first=False):
        if coef == 0:
            return ""
        if first:
            if coef == 1 and var:
                return var
            elif coef == -1 and var:
                return f"-{var}"
            else:
                return f"{coef}{var}"
        else:
            if coef == 1 and var:
                return f" + {var}"
            elif coef == -1 and var:
                return f" - {var}"
            elif coef > 0:
                return f" + {coef}{var}"
            else:
                return f" - {abs(coef)}{var}"

    eq = fmt(a, "x²", first=True) + fmt(b, "x") + fmt(c, "")
    print(f"{eq} = 0   Δ = {delta}")  

In [4]:
weights = [1/4, 1/6, 1/3, 1/4]
values  = [-2, -1, 0, 1]

print(calculate_expectation(weights, values))
print(calculate_variance(weights, values))    

-0.41666666666666663
1.2430555555555556


In [5]:
# Case 1.
def generate_discrete_case1():
    E       = [i for i in range(-9, 10) if i != 0]     # 18 elements
    E_small = [1, 2, 3]                                 # 3 elements

    # Probability of full E, equally
    probs_E     = [1/len(E)]       * len(E)             # 1/18 each
    probs_small = [1/len(E_small)] * len(E_small)       # 1/3 each

    # Generate Discrete sample a, b from full E
    a = generate_discrete_sample(E, probs_E)
    b = generate_discrete_sample(E, probs_E)

    # Generate Discrete sample e from small E
    e = generate_discrete_sample(E_small, probs_small)

    # c = (b**2 + e) / (4|a|)  — use abs(a) so the denominator is always positive
    c = (b**2 + e) / (4 * abs(a))

    # Prevent sign error: when a < 0, flip c so that c > b²/(4a) still holds
    if a < 0:
        c = -c

    delta = b**2 - 4 * a * c

    return a, b, c, delta

In [6]:
for _ in range(10):
    A, B, C, delta = generate_discrete_case1()
    display_equation(A, B, C, delta)

-4x² - 8x - 4.1875 = 0   Δ = -3.0
-2x² - 8x - 8.25 = 0   Δ = -2.0
-x² + 2x - 1.25 = 0   Δ = -1.0
-2x² + 6x - 4.625 = 0   Δ = -1.0
-8x² - 6x - 1.2188 = 0   Δ = -3.0
-4x² - 6x - 2.3125 = 0   Δ = -1.0
-6x² - 3x - 0.4583 = 0   Δ = -2.0
-6x² - 3x - 0.5 = 0   Δ = -3.0
-8x² - 8x - 2.0938 = 0   Δ = -3.0
-9x² + 5x - 0.7222 = 0   Δ = -1.0


In [7]:
# Case 2.
def generate_discrete_case2():
    E = [i for i in range(-9, 10) if i != 0 ]
    E_positive = [i for i in range(1, 10)]
    probs_E = [1/len(E)] * len(E)
    
    probs_E_positive = [1/2, 1/36, 1/36, 1/6, 1/36, 1/36, 1/36, 1/36, 1/6]

    # generate x0

    ## e ~ U(E)
    e = generate_discrete_sample(E, probs_E)
    ## l ~ X ~ U(E+)
    l = generate_discrete_sample(E_positive, probs_E_positive)
    
    x0 = e / math.sqrt(l)

    # calculate a, b, c
    a = 1
    b = -2*a*x0
    c = a*x0**2

    delta = b**2 - 4 * a * c
    return a, b, c, delta

In [8]:
for _ in range(10):
    A, B, C, delta = generate_discrete_case2()
    display_equation(A, B, C, delta)

x² - 3.3333x + 2.7778 = 0   Δ = 0.0
x² + 16.0x + 64.0 = 0   Δ = 0.0
x² + 3.7796x + 3.5714 = 0   Δ = 0.0
x² - 5.0x + 6.25 = 0   Δ = 0.0
x² - 2.1213x + 1.125 = 0   Δ = 0.0
x² + 4.0x + 4.0 = 0   Δ = 0.0
x² - 5.3333x + 7.1111 = 0   Δ = 0.0
x² + 5.3333x + 7.1111 = 0   Δ = 0.0
x² + 4.0x + 4.0 = 0   Δ = 0.0
x² - 2.2678x + 1.2857 = 0   Δ = 0.0


In [9]:
# Case 3.
def generate_discrete_case3():
    E        = [i for i in range(-9, 10) if i != 0]
    E_positive = [i for i in range(1, 10)]
    E_small  = [i for i in range(-3, 4) if i != 0]

    # Probability of full E, equally
    probs_E         = [1/len(E)]          * len(E)
    probs_E_positive = [1/len(E_positive)] * len(E_positive)
    probs_E_small   = [1/len(E_small)]    * len(E_small)

    # given probability, P(Z=1)=1/2, P(Z=other)=1/(2*17)
    probs_Z = [1/2 if i == 1 else 1/(2*17) for i in E]

    # randomly select sub-case
    case_prob = generate_discrete_sample([1, 2], [1/2, 1/2])

    if case_prob == 1:
        # Case 3.1 — rational roots x1=h/l, x2=k/l
        h   = generate_discrete_sample(E, probs_E)
        k   = generate_discrete_sample(E, probs_E)

        # l from Z which given
        ell = generate_discrete_sample(E, probs_Z)

        x1 = h / ell
        x2 = k / ell

        # calcalate a, b, c
        a = 1
        b = -(x1 + x2)
        c = x1 * x2

    else:
        # Case 3.2 — roots are x = (-h ± e√p) / l
        # Clear denominators → ℓ²x² + 2hℓx + (h² - pe²) = 0
        h = generate_discrete_sample(E, probs_E)

        # Sampling from set E[-3,-2,-1,1,2,3]
        l = generate_discrete_sample(E_small, probs_E_small)

        # Sampling from set E[1....9]
        e = generate_discrete_sample(E_positive, probs_E_positive)
        p = generate_discrete_sample(E_positive, probs_E_positive)

        # Calculate from the given formula
        a = l ** 2
        b = 2 * h * l
        c = h ** 2 - p * e ** 2

    delta = b**2 - 4 * a * c

    return a, b, c, delta

In [10]:
for _ in range(10):
    A, B, C, delta = generate_discrete_case3()
    display_equation(A, B, C, delta)

x² + 10x - 103 = 0   Δ = 512
x² - x - 20.0 = 0   Δ = 81.0
x² - 7.0x + 6.0 = 0   Δ = 25.0
x² - 0.4x - 1.4 = 0   Δ = 5.76
x² - 3.0x + 0.8889 = 0   Δ = 5.4444
x² - 1.7143x + 0.551 = 0   Δ = 0.7347
4x² + 36x + 54 = 0   Δ = 432
9x² - 36x - 356 = 0   Δ = 14112
x² + 18x - 47 = 0   Δ = 512
x² - 2.0x - 35.0 = 0   Δ = 144.0
